Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
lb = (np.arange(10) == train_labels[0:5, None].astype(np.float32))
print(lb)
train_labels[5]

[[False False False False  True False False False False False]
 [False False False False False False False False False  True]
 [False False False False False False  True False False False]
 [False False  True False False False False False False False]
 [False False False False False False False  True False False]]


3

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

### Logistic Regression through Linear Regression and Softmax without Regularisation

In [5]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run this computation and iterate:

In [6]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 21.230494
Training accuracy: 10.0%
Validation accuracy: 12.3%
Loss at step 100: 2.280164
Training accuracy: 72.4%
Validation accuracy: 71.1%
Loss at step 200: 1.855188
Training accuracy: 75.2%
Validation accuracy: 73.5%
Loss at step 300: 1.611606
Training accuracy: 76.5%
Validation accuracy: 74.5%
Loss at step 400: 1.445386
Training accuracy: 77.3%
Validation accuracy: 75.0%
Loss at step 500: 1.321348
Training accuracy: 78.1%
Validation accuracy: 75.2%
Loss at step 600: 1.223665
Training accuracy: 78.6%
Validation accuracy: 75.4%
Loss at step 700: 1.144173
Training accuracy: 79.2%
Validation accuracy: 75.7%
Loss at step 800: 1.077906
Training accuracy: 79.6%
Validation accuracy: 75.9%
Test accuracy: 82.9%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

### Stochastic Gradient Descent with linear activation function and softmax

In [7]:

batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [8]:
256 % (train_labels.shape[0] - batch_size)

256

In [9]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Offset -- %d Minibatch loss at step %d: %f" % (offset, step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Offset -- 0 Minibatch loss at step 0: 18.864063
Minibatch accuracy: 14.8%
Validation accuracy: 16.6%
Offset -- 64000 Minibatch loss at step 500: 1.836201
Minibatch accuracy: 71.1%
Validation accuracy: 75.2%
Offset -- 128000 Minibatch loss at step 1000: 1.572161
Minibatch accuracy: 73.4%
Validation accuracy: 76.9%
Offset -- 192000 Minibatch loss at step 1500: 0.811520
Minibatch accuracy: 78.1%
Validation accuracy: 77.6%
Offset -- 56128 Minibatch loss at step 2000: 0.964155
Minibatch accuracy: 78.1%
Validation accuracy: 77.8%
Offset -- 120128 Minibatch loss at step 2500: 0.965130
Minibatch accuracy: 80.5%
Validation accuracy: 78.4%
Offset -- 184128 Minibatch loss at step 3000: 0.990276
Minibatch accuracy: 77.3%
Validation accuracy: 78.8%
Test accuracy: 85.9%


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

### One Hidden Layer NN (1024 units) with Relu and fancy weight initialisation with SGD and softmax

In [178]:
# with one Relu hidden layer
batch_size = 128
L1 = 4096
L2 = 10

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, L1], stddev=np.sqrt(2/(image_size*image_size))))
  biases1 = tf.Variable(tf.ones([L1])/10)
  weights2 = tf.Variable(tf.truncated_normal([L1, L2], stddev=np.sqrt(2/L1)))
  biases2 = tf.Variable(tf.zeros([L2]))
  
  # Training computation.
  Y1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  logits = tf.matmul(Y1, weights2) + biases2
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  # Validation Computation
  Y1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  logits_valid = tf.matmul(Y1_valid, weights2) + biases2
  # Test Computation
  Y1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  logits_test = tf.matmul(Y1_test, weights2) + biases2 
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(logits_valid)
  test_prediction = tf.nn.softmax(logits_test)
# lr = 0.1
# without touching weights, the best option for biases as per accuracy (90.6%) is to use tf.Variable(tf.ones([L1])/10) for biases for Relu 
# and tf.Variable(tf.zeros([L2])) for non Relu functions
# accuracy jumps from 90.6% to 92.3% only by adding to weights1 sttdev=np.sqrt(2/n) where n = # of neurons in prev
# layer. In case of Relu here I used n = # of features since that's what Relu has as input in this case.
# Accuracy jumps from 92.3% to 93.1% by adding stddev=np.sqrt(2/L1) to weights2
# acc goes to 94.2% when using lr = 0.5 with GD

In [179]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.492136
Minibatch accuracy: 11.7%
Validation accuracy: 31.9%
Minibatch loss at step 500: 0.514753
Minibatch accuracy: 82.8%
Validation accuracy: 85.5%
Minibatch loss at step 1000: 0.555979
Minibatch accuracy: 82.0%
Validation accuracy: 86.5%
Minibatch loss at step 1500: 0.375810
Minibatch accuracy: 92.2%
Validation accuracy: 88.0%
Minibatch loss at step 2000: 0.314721
Minibatch accuracy: 89.1%
Validation accuracy: 88.4%
Minibatch loss at step 2500: 0.424420
Minibatch accuracy: 90.6%
Validation accuracy: 89.0%
Minibatch loss at step 3000: 0.416072
Minibatch accuracy: 89.1%
Validation accuracy: 89.0%
Test accuracy: 94.0%


In [10]:
import pandas as pd
tr_resh_pd = pd.DataFrame(train_dataset.reshape(200000, -1)).drop_duplicates()
ts_resh_pd = pd.DataFrame(test_dataset.reshape(10000, -1)).drop_duplicates()
vl_resh_pd = pd.DataFrame(valid_dataset.reshape(10000, -1)).drop_duplicates()

tr_resh_pd = pd.DataFrame(train_dataset.reshape(200000, -1))
ts_resh_pd = pd.DataFrame(test_dataset.reshape(test_dataset.shape[0], -1))
vl_resh_pd = pd.DataFrame(valid_dataset.reshape(valid_dataset.shape[0], -1))

ts_in_tr = tr_resh_pd.merge(ts_resh_pd)
vl_in_tr = tr_resh_pd.merge(vl_resh_pd)
vl_in_ts = ts_resh_pd.merge(vl_resh_pd)

print("Duplicated in train:",train_dataset.shape[0]-tr_resh_pd.shape[0])
print("Duplicated in test:", test_dataset.shape[0]-ts_resh_pd.shape[0])
print("Duplicated in valid:",valid_dataset.shape[0]-vl_resh_pd.shape[0])
print("Overlapping observations between train and test: %s" % ts_in_tr.shape[0])
print("Overlapping observations between train and validation: %s" % vl_in_tr.shape[0])
print("Overlapping observations between validation and test: %s"  %vl_in_ts.shape[0])

Duplicated in train: 0
Duplicated in test: 0
Duplicated in valid: 0
Overlapping observations between train and test: 283979
Overlapping observations between train and validation: 219700
Overlapping observations between validation and test: 14363


In [4]:
%%time
3 ** 12

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.48 µs


531441